# Integrative analysis of pathway deregulation in obesity #

## Python implementation

### Steps (according to the paper)

1. Probes containing missing values are excluded from the analysis. 

2. Probes are mapped to Entrez ID labels if they are available in the associated platform. Otherwise the David portal is used to convert the available labels to Entrez ID labels. 

3. Values corresponding to raw expression counts or gene expression intensity are log2 transformed (if necessary). 

4. Probes mapping to the same Entrez ID label are averaged out. 

5. Probes that cannot be mapped to a unique Entrez ID label are excluded from the analysis, as well as those that cannot be mapped to any Entrez ID label at all. 

6. We apply a simple L1 normalization in linear space, imposing that the sum of expression of all genes is constant among samples. After these steps, each data set or batch is represented by a single expression matrix X. Each entry Xi j represents the log2 of the expression intensity of gene i in sample j.

### Imports

In [1]:
# Import std libraries
import os
from operator import itemgetter 
import re

# Import third party
import numpy as np
import pandas as pd
import GEOparse

# Set logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logging.getLogger("GEOparse").setLevel(logging.WARNING)

### Load Dataset

Download the dataset (if needed) and load it.

Some GEOparse names:
- DataSet (GDS)
- Series (GSE)
- Platform (GPL)
- Samples (GSM)

In [2]:
def load_dataset(dataset_id):
    """
    Load the dataset from disk (or download it if it does not exists)
    Arguments:
    - dataset_id: the ID of the dataset to load
    
    Output:
    - GSE object (GEOparse Series)
    """
    path = "./" + dataset_id + "_family.soft.gz"
    if os.path.exists(path):
        # Load from an existing file
        print("- Loading from", path)
        gse = GEOparse.get_GEO(filepath=path)
    else:
        # Download GSE and load it
        print("- Downloading", dataset_id)
        gse = GEOparse.get_GEO(geo=dataset_id, destdir="./")
    return gse

dataset_id = "GSE27949"
gse = load_dataset(dataset_id)
print("- Dataset loaded")

- Loading from ./GSE27949_family.soft.gz


/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:502: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  gpls[entry_name] = parse_GPL(data_group, entry_name)


- Dataset loaded


### Get Info

Get some useful info and statistics from our data.

We're going to extract:
- number of platforms
- number of samples
- dimension of each sample

In [3]:
# data_frames contains the data-frame of each sample
# samples_name is a list which contains the name associated to each dataframe
data_frames = []
samples_information = []
for gsm_name, gsm in gse.gsms.items():
    #print(gsm.metadata)
    email = gsm.metadata['contact_email'][0]
    bmi = re.findall('\d+', gsm.metadata['characteristics_ch1'][1])
    bmi = float(bmi[0])
    if bmi >= 30:
        samples_information.append(gsm_name + "_" + "OU")
        data_frames.append(gsm.table)
    elif bmi <= 24.9:
        samples_information.append(gsm_name + "_" + "LU")
        data_frames.append(gsm.table)
    else:
        continue

print("- Number of platforms", len(gse.gpls.items()))
print("- Number of samples", len(data_frames))
print("- Dimension of each sample (assuming are all the same)", data_frames[0].shape)
print('\nExample of a sample dataframe:')
data_frames[0].head()
print(data_frames[0].shape)
print(samples_information)

- Number of platforms 1
- Number of samples 23
- Dimension of each sample (assuming are all the same) (54675, 2)

Example of a sample dataframe:
(54675, 2)
['GSM691142_OU', 'GSM691122_OU', 'GSM691127_OU', 'GSM691143_LU', 'GSM691130_OU', 'GSM691136_LU', 'GSM691146_OU', 'GSM691123_OU', 'GSM691128_OU', 'GSM691129_OU', 'GSM691145_OU', 'GSM691132_OU', 'GSM691152_LU', 'GSM691153_LU', 'GSM691138_OU', 'GSM691125_OU', 'GSM691124_OU', 'GSM691151_LU', 'GSM691131_OU', 'GSM691126_OU', 'GSM691154_LU', 'GSM691134_OU', 'GSM691144_OU']


### Filter data #1

We're going to:
- Remove probes from the mapper, mapping to multiple Entrez IDs
- Construct a Python dictionary containing the valid probes and their Entrez ID

In [4]:
def create_mapper(meta_data_tables):
    """
    Returns a python dictionary that represents our mapper object
    Important: not all probs_id are mapped to an ENTREZ_GENE_ID
    probs_id without an enterez_id are not added to the dictionary
    """
    mapper = {}
    for df in meta_data_tables:
        for index, row in df.iterrows():
            probs_id = row['ID']
            
            if probs_id in mapper and mapper[probs_id] != row['ENTREZ_GENE_ID']:
                # Multiple enterez id for the same probs
                # Set their value to None to invalid them
                # Elements set to "None" are then removed
                mapper[probs_id] = None
                
            if probs_id not in mapper and not pd.isnull(row['ENTREZ_GENE_ID']):
                mapper[probs_id] = row['ENTREZ_GENE_ID']
            
    # Remove invalid mapping (value = None)
    # (Some of the probes are linked with multiple numbers (enterez_id ?) using /// as separator)
    filtered_mapper = {k:v for k,v in mapper.items() if v != None and '/' not in v}
    
    return filtered_mapper

meta_data_tables = []
for gpl_name, gpl in gse.gpls.items():
    meta_data_tables.append(gpl.table)

mapper = create_mapper(meta_data_tables)
print("- Mapper loaded", len(mapper))

- Mapper loaded 41834


### Filter data #2: 
#### Remove rows without a matching enterez_id
Intuition: 
1. Convert the dictionary **mapper** into a pandas' DataFrame (**mapper_df**).  
2. Use a SQL-like inner join to merge **mapper_df** with the existing pandas' DataFrame.  
Inner join creates a new Dataframe with *only* the matching rows.

References:
- https://www.w3schools.com/sql/sql_join_inner.asp
- https://pandas.pydata.org/pandas-docs/stable/merging.html

In [5]:
# Convert mapper to a Pandas Dataframe with two columns (probs, enterez_id)
mapper_df = pd.DataFrame.from_dict(mapper, orient='index')
mapper_df.index.name = 'ID_REF'
mapper_df.columns = ['ENTREZ_GENE_ID']
print("\nExample of the obtained Pandas DataFrame:")

# Create a mapper (sample_id, person_id)
#mapper_sample_person = pd.DataFrame(samples_label)
#mapper_sample_person = mapper_sample_person.set_index([samples_name])
#mapper_sample_person = mapper_sample_person.transpose()
#mapper_sample_person.head()


Example of the obtained Pandas DataFrame:


### Creation of a Dataframe Entrez ID - Value: genes common to all array
For each sample:
1. We convert probes' values in **log2(values)**
2. Rows with the same **entrez_id** are merged together using the average (probes mapping to the same Entrez ID are averaged out).

In [6]:
# Update each df object inside the list 'data_frames' with the dataframe with only matching probs_id
# For each sample's dataframes:
# - log2
# - only matching entrez_id
# - L1 normalization
data_frames_entrez = []
for df in data_frames:
    df = pd.merge(df, mapper_df, how='inner', left_on=['ID_REF'], right_index=True, sort=False)
    df = df.groupby('ENTREZ_GENE_ID').mean()
    df['VALUE'] = 2**df['VALUE']
    df.VALUE = df.VALUE / df.VALUE.sum()
    df['VALUE'] = np.log2(df['VALUE'])
    data_frames_entrez.append(df)

""" part that has to deal with multiple sample for the same person
# ok, now we have to concat all the dataframes of each person
data_frames_per_person = []

# remove repeated elements from samples_label
unique_person_label = list(set(samples_label))

for person_label in unique_person_label:
    
    # get index of repeated elements in samples_label that has the value = person_label
    index_to_merge = [i for i, x in enumerate(samples_label) if x == person_label]
    
    # get elements from data_frames_entrez given the index in index_to_merge
    arrays = list(itemgetter(*index_to_merge)(data_frames_entrez))
    
    # if arrays contains the same subset of entrez_id, do the mean
    all_gene_person = pd.concat(arrays).groupby('ENTREZ_GENE_ID').mean()
    
    # save the dataset with all the 5 arrays inside data_frames_per_person
    data_frames_per_person.append(all_gene_person)
"""

# concat data_frames by columns and use the people's label as index
merged_entrez_value_df = pd.concat(data_frames_entrez, axis=1, keys=samples_information)
print("size before any removal", merged_entrez_value_df.shape)

# remove gene with at least 1 missing value
merged_entrez_value_df = merged_entrez_value_df.dropna(axis=0, how='any')
print("size after removing genes with missing value", merged_entrez_value_df.shape)

# remove disturbing index "VALUE"
merged_entrez_value_df.columns = merged_entrez_value_df.columns.droplevel([1])

print("\nExample of the obtained merged data frame:")
merged_entrez_value_df.head()

size before any removal (20486, 23)
size after removing genes with missing value (20486, 23)

Example of the obtained merged data frame:


,GSM691142_OU,GSM691122_OU,GSM691127_OU,GSM691143_LU,GSM691130_OU,GSM691136_LU,GSM691146_OU,GSM691123_OU,GSM691128_OU,GSM691129_OU,...,GSM691153_LU,GSM691138_OU,GSM691125_OU,GSM691124_OU,GSM691151_LU,GSM691131_OU,GSM691126_OU,GSM691154_LU,GSM691134_OU,GSM691144_OU
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
1,-16.656284,-16.467288,-16.644595,-16.672241,-16.616464,-16.946165,-16.253565,-16.911198,-17.268633,-17.002634,...,-16.941852,-17.034051,-17.102832,-16.751461,-16.978019,-16.991578,-17.002924,-17.219474,-17.365789,-17.142668
10,-16.941340,-17.058493,-16.668810,-17.026982,-16.587789,-17.313423,-17.003948,-16.863739,-17.370265,-17.199917,...,-16.768056,-17.188533,-16.889089,-17.086915,-17.200414,-16.900854,-17.292170,-16.936701,-17.048590,-17.194503
100,-15.335082,-15.143535,-14.966167,-15.173494,-14.668714,-15.262088,-14.900857,-15.211285,-15.049138,-15.427917,...,-14.530176,-14.857492,-14.978559,-15.229336,-15.602385,-14.842609,-15.146958,-14.924284,-15.921903,-15.019982
1000,-16.957133,-16.888659,-17.109824,-17.145079,-17.317420,-17.175075,-16.833997,-16.882759,-17.470116,-17.377273,...,-17.191087,-17.595929,-17.137453,-17.238383,-17.253288,-17.556044,-17.451344,-17.332248,-17.216601,-17.425703
10000,-15.973645,-15.917606,-15.773719,-16.009637,-15.943373,-15.714638,-15.984475,-16.035525,-16.155986,-16.133969,...,-16.016150,-16.100682,-16.029792,-15.821668,-16.253650,-15.629414,-16.177538,-15.829951,-16.789228,-15.988975


In [7]:
merged_entrez_value_df.to_pickle("data/GSE27949_table.pkl")